<a href="https://colab.research.google.com/github/Soumikcdsfjshdhfjhdsfdshfhdshdshj/CnA/blob/main/Dynamic%20Parking%20Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Parking Pricing Models:

This notebook implements and explains two dynamic parking pricing models:

- **Model 1:** Baseline Linear Pricing
- **Model 2:** Demand-Based Pricing

Each section includes code, comments, and explanations of the demand function, assumptions, and price adjustment logic.

## 1. Setup and Data Preparation

We install required libraries, load the dataset, and prepare the data for both models.


In [ ]:
# Install necessary packages
!pip install pathway panel bokeh --quiet

import pandas as pd
import numpy as np
import pathway as pw
import panel as pn
import bokeh.plotting
from bokeh.models import ColumnDataSource, HoverTool, DatetimeTickFormatter

# Enable Panel extension for interactive plots
pn.extension()

# Load dataset
df = pd.read_csv("/content/dataset.csv")

# Combine date and time into a single timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Sort by time for streaming simulation
df = df.sort_values('Timestamp')

# Add/ensure all required columns exist
np.random.seed(42)  # For reproducible competitor prices
df['QueueLength'] = df.get('QueueLength', 0.0)
df['TrafficLevel'] = df.get('TrafficLevel', 0.5)
df['IsSpecialDay'] = df.get('IsSpecialDay', False)
df['VehicleType'] = df.get('VehicleType', 'CAR')
df['CompetitorPrice'] = 10 + np.random.normal(loc=0.0, scale=1.5, size=len(df)).round(2)

# Save for streaming input
df[['Timestamp', 'SystemCodeNumber', 'Occupancy', 'Capacity',
    'QueueLength', 'TrafficLevel', 'IsSpecialDay', 'VehicleType', 'CompetitorPrice']].to_csv("parking_stream.csv", index=False)


## 2. Pathway Schema and Data Stream

We define the schema for our data and load it as a stream for real-time simulation.


In [ ]:
# Define schema for Pathway streaming
class PricingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: float
    TrafficLevel: float
    IsSpecialDay: bool
    VehicleType: str
    CompetitorPrice: float

# Load data as a stream
data = pw.demo.replay_csv("parking_stream.csv", schema=PricingSchema, input_rate=1000)

# Add parsed datetime columns for easier processing
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S"),
    day = data.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S").dt.strftime("%Y-%m-%dT00:00:00")
)


## 3. Model 1: Baseline Linear Pricing

### Demand Function

- **Formula:**  
- Price(t+1) = Price(t) + a*(Occupancy/Capacity)

- Price increases linearly with occupancy rate.

### Assumptions

- Each lot is priced independently.
- Base price resets daily.
- Occupancy is the only factor influencing price.

### Price Adjustment

- Higher occupancy → higher price, encouraging turnover and space availability.


In [ ]:
# Model 1: Baseline Linear Pricing
model1_state = {}

def baseline_price_model(occupancy, capacity, system_code, timestamp):
    """
    Updates price linearly based on occupancy rate.
    Daily price resets for each lot.
    """
    global model1_state
    day = timestamp.date()
    if system_code not in model1_state:
        model1_state[system_code] = {}
    if day not in model1_state[system_code]:
        model1_state[system_code][day] = 10.0  # Daily reset base price
    prev = model1_state[system_code][day]
    rate = occupancy / capacity
    price = prev + 0.5 * rate  # Linear increment
    model1_state[system_code][day] = price
    return round(price, 2)

# Apply Model 1 to the data stream
model1_output = data_with_time.with_columns(
    price=pw.apply(baseline_price_model,
                   pw.this.Occupancy,
                   pw.this.Capacity,
                   pw.this.SystemCodeNumber,
                   pw.this.t),
    competitor=pw.this.CompetitorPrice
)


### Model 1 Visualization

The plot below shows our price (solid) and competitor price (dashed) for each lot over time.


In [ ]:
def visualize(pricing_data, model_name):
    """
    Visualizes pricing data for each lot, comparing our price and competitor price.
    """
    df = pw.debug.table_to_pandas(pricing_data)
    if df.empty:
        print("No data to plot.")
        return

    df['t'] = pd.to_datetime(df['t'])
    df = df.sort_values("t")
    lots = sorted(df['SystemCodeNumber'].unique())
    plots = []

    for lot in lots:
        sub = df[df['SystemCodeNumber'] == lot]
        source = ColumnDataSource({
            't': sub['t'],
            'price': sub['price'],
            'competitor': sub['competitor'],
            'SystemCodeNumber': sub['SystemCodeNumber']
        })

        fig = bokeh.plotting.figure(
            height=300,
            width=1100,
            title=f"{model_name} – Lot {lot}",
            x_axis_type="datetime",
            x_axis_label="Time",
            y_axis_label="Price",
            toolbar_location="right",
            tools="pan,wheel_zoom,box_zoom,reset,save"
        )

        fig.line("t", "price", source=source, line_width=2, color="navy", legend_label="Our Price")
        fig.line("t", "competitor", source=source, line_width=2, color="gray", line_dash="dashed", legend_label="Competitor")

        fig.add_tools(HoverTool(
            tooltips=[
                ("Time", "@t{%F %T}"),
                ("Our Price", "@price"),
                ("Competitor", "@competitor"),
                ("Lot", "@SystemCodeNumber")
            ],
            formatters={"@t": "datetime"},
            mode="vline"
        ))

        fig.xaxis.formatter = DatetimeTickFormatter(
            hours="%H:%M",
            days="%d %b",
            months="%b %Y",
            milliseconds="%H:%M:%S.%3N"
        )

        fig.legend.location = "top_left"
        fig.legend.click_policy = "hide"
        plots.append(fig)

    return pn.Column(*plots, sizing_mode="stretch_width", scroll=True)


viz_model1 = visualize(model1_output, "Model 1: Baseline Pricing")
pn.Tabs(("Model 1", viz_model1)).servable()


Tabs
    [0] Column(scroll=True, sizing_mode='stretch_width')
        [0] Bokeh(figure)
        [1] Bokeh(figure)
        [2] Bokeh(figure)
        [3] Bokeh(figure)
        [4] Bokeh(figure)
        [5] Bokeh(figure)
        [6] Bokeh(figure)
        [7] Bokeh(figure)
        [8] Bokeh(figure)
        [9] Bokeh(figure)
        [10] Bokeh(figure)
        [11] Bokeh(figure)
        [12] Bokeh(figure)
        [13] Bokeh(figure)

## 4. Model 2: Demand-Based Pricing

### Demand Function

- **Formula:**  
 Demand = α·(Occupancy/ Capacity)
 +β·QueueLength−γ·Traffic+δ·IsSpecialDay+ε·VehicleTypeWeight
- Use this demand value to adjust prices:
 Pricet = BasePrice · (1 + λ · NormalizedDemand)
- Price is scaled by normalized demand and clamped between 0.5x and 2x the base price.

### Assumptions

- All features are available and accurate.
- Weights for features are based on intuition and can be tuned.
- Each lot is priced independently.
- Base price resets daily.

### Price Adjustment

- Higher occupancy, queue, special day, or larger vehicles → higher price.
- Higher traffic (congestion) reduces price.
- Price is bounded to avoid extreme values.


In [ ]:
# Model 2: Demand-Based Pricing
model2_state = {}
vehicle_weights = {"CAR": 1.0, "BIKE": 0.7, "BUS": 1.5, "TRUCK": 1.8, "OTHER": 1.0}

def demand_based_model(occ, cap, sys, ts, q, traffic, special, vtype):
    """
    Calculates price based on a weighted demand function.
    Clamps price between 0.5x and 2x base price.
    """
    global model2_state
    day = ts.date()
    if sys not in model2_state:
        model2_state[sys] = {}
    if day not in model2_state[sys]:
        model2_state[sys][day] = 10.0  # Daily reset base price
    base = model2_state[sys][day]
    occ_rate = occ / cap
    special = 1 if special else 0
    vw = vehicle_weights.get(vtype.upper(), 1.0)
    α, β, γ, δ, ε = 1.0, 0.5, 1.0, 0.3, 0.8
    demand = α*occ_rate + β*q - γ*traffic + δ*special + ε*vw
    norm = np.clip((demand - 0.5) / 3, 0, 1)
    λ = 0.8
    price = base * (1 + λ * norm)
    return round(np.clip(price, base * 0.5, base * 2.0), 2)

# Apply Model 2 to the data stream
model2_output = data_with_time.with_columns(
    price=pw.apply(demand_based_model,
                   pw.this.Occupancy,
                   pw.this.Capacity,
                   pw.this.SystemCodeNumber,
                   pw.this.t,
                   pw.this.QueueLength,
                   pw.this.TrafficLevel,
                   pw.this.IsSpecialDay,
                   pw.this.VehicleType),
    competitor=pw.this.CompetitorPrice
)


### Model 2 Visualization

The plot below shows our demand-based price (solid) and competitor price (dashed) for each lot over time.


In [ ]:

viz_model2 = visualize(model2_output, "Model 2: Demand-Based Pricing")
pn.Tabs(("Model 2", viz_model2)).servable()


Tabs
    [0] Column(scroll=True, sizing_mode='stretch_width')
        [0] Bokeh(figure)
        [1] Bokeh(figure)
        [2] Bokeh(figure)
        [3] Bokeh(figure)
        [4] Bokeh(figure)
        [5] Bokeh(figure)
        [6] Bokeh(figure)
        [7] Bokeh(figure)
        [8] Bokeh(figure)
        [9] Bokeh(figure)
        [10] Bokeh(figure)
        [11] Bokeh(figure)
        [12] Bokeh(figure)
        [13] Bokeh(figure)

## 5. Notes on Competition

- **Competitor prices** are included for comparison in both models.
- **Model 3 (Competitive Pricing)** is not implemented, so our models do not directly react to competitor prices.
- If implemented, Model 3 would allow price adjustment based on nearby lot prices and proximity.

---

## 6. Summary

- **Model 1** uses a simple linear occupancy-based price update.
- **Model 2** incorporates multiple demand factors for more dynamic pricing.
- Both models are visualized and compared to simulated competitor prices.

---

*End of notebook.*
